In [1]:
import pandas as pd

data = pd.read_parquet('../cache/btc.parquet')
data.head()

,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [2]:
data.columns

Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [3]:
# drop NaN values
data.dropna(inplace=True)

In [4]:
data['Timestamp'] = data['Timestamp'].astype('int64')

# get data from ranges 2019-01-01 to 2023-12-31
timeStamp1 = pd.Timestamp('2018-12-31').value // 10**9
timeStamp2 = pd.Timestamp('2024-01-02').value // 10**9

data= data[(data['Timestamp'] >= timeStamp1) & (data['Timestamp'] <= timeStamp2)]

In [5]:
# sample every 15th row
data = data.iloc[::15]
data.head()

,Timestamp,Open,High,Low,Close,Volume
3679619,1546214400,3811.45,3813.76,3808.00,3811.61,28.416599
3679634,1546215300,3819.92,3819.92,3819.92,3819.92,0.000000
3679649,1546216200,3816.89,3816.89,3816.89,3816.89,0.416601
3679664,1546217100,3827.30,3827.30,3823.50,3824.20,7.574298
3679679,1546218000,3808.66,3808.66,3805.97,3805.97,0.283735


In [6]:
data = data.drop(columns=['Volume', 'High', 'Low', 'Close'])

In [7]:
data['Open_shifted'] = data['Open'].shift(-2)

# Calculate the percentage change
data['Open_pct_change'] = ((data['Open_shifted'] - data['Open']) / data['Open']) * 100

data.head()

,Timestamp,Open,Open_shifted,Open_pct_change
3679619,1546214400,3811.45,3816.89,0.142728
3679634,1546215300,3819.92,3827.30,0.193198
3679649,1546216200,3816.89,3808.66,-0.215621
3679664,1546217100,3827.30,3809.77,-0.458025
3679679,1546218000,3808.66,3815.83,0.188255


In [8]:
data = data.drop(columns=['Open', 'Open_shifted'])

timeStamp1 = pd.Timestamp('2019-01-01').value // 10**9
timeStamp2 = pd.Timestamp('2024-01-01 00:00:00').value // 10**9

data = data[(data['Timestamp'] >= timeStamp1) & (data['Timestamp'] < timeStamp2)]
data.reset_index(drop=True, inplace=True)
data.head()

,Timestamp,Open_pct_change
0,1546300800,-0.226363
1,1546301700,-2.578560
2,1546302600,-1.509301
3,1546303500,0.781614
4,1546304400,-0.291943


In [9]:
import datetime

date_time = datetime.datetime.utcfromtimestamp(data['Timestamp'].iloc[0])
print(f"first: {date_time}")
date_time = datetime.datetime.utcfromtimestamp(data['Timestamp'].iloc[1])
print(f"second: {date_time}")
date_time = datetime.datetime.utcfromtimestamp(data['Timestamp'].max())
print(f"last: {date_time}")

first: 2019-01-01 00:00:00
second: 2019-01-01 00:15:00
last: 2023-12-31 23:45:00


/tmp/ipykernel_1779/2418492704.py:3: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date_time = datetime.datetime.utcfromtimestamp(data['Timestamp'].iloc[0])
/tmp/ipykernel_1779/2418492704.py:5: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date_time = datetime.datetime.utcfromtimestamp(data['Timestamp'].iloc[1])
/tmp/ipykernel_1779/2418492704.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date_time = datetime.datetime.utc

In [10]:
data.to_parquet("../cache/btc_cleaned.parquet")